In [4]:
# Import checkpoint
from __future__ import division
import tensorflow as tf
import numpy as np
import pandas as pd


# Load dataset
train_path = "pickle_data/n2fft_05_dataframe_full_train.pkl"
test_path = "pickle_data/n2fft_05_dataframe_full_test.pkl"

# Training set
train_df = pd.read_pickle(train_path)
train_features = np.stack(train_df['FFT_DATA'].values)
train_labels = train_df['MARK'].values.astype(int)
del train_df

print('Loaded Train Data: features ', train_features.shape, ' labels', train_labels.shape)

# Create weight for training
train_weight = np.ones(train_features.shape[0])
train_weight[train_labels == 1] = 4

# Test set
test_df = pd.read_pickle(test_path)
test_features = np.stack(test_df['FFT_DATA'].values)
test_labels = test_df['MARK'].values.astype(int)
del test_df

print('Loaded Test Data: features ', test_features.shape, ' labels', test_labels.shape)

# Normalize data
mean_train = np.mean(train_features, axis=0)
std_train = np.std(train_features, axis=0)
train_features = (train_features - mean_train)/std_train
test_features = (test_features - mean_train)/std_train

# Specify that all features have real-value data
feature_columns = [tf.feature_column.numeric_column("x", shape=train_features.shape[1])]
weight_column = tf.feature_column.numeric_column("weight")

# Build 2 layer DNN with 128, 64, 1 units respectively
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                        hidden_units=[256, 128],
                                        n_classes=2,
                                        model_dir="model_data/spindle_estimator",
                                        optimizer=tf.train.AdamOptimizer(learning_rate=0.001),
                                        weight_column=weight_column,
                                        dropout=0.4)

#    # Define the training inputs
#    train_input_fn = tf.estimator.inputs.numpy_input_fn(
#        x={"x": train_features, "weight": train_weight},
#        y=train_labels,
#        num_epochs=10,
#        shuffle=True,
#        batch_size=64
#    )
#
#    # Train model
#    classifier.train(input_fn=train_input_fn)
#
# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": test_features, "weight": np.ones(test_features.shape[0])},
    y=test_labels,
    num_epochs=1,
    shuffle=False)

# Evaluate
metrics = classifier.evaluate(input_fn=test_input_fn)
print(metrics)
    
    

Loaded Train Data: features  (2895900, 20)  labels (2895900,)
Loaded Test Data: features  (242400, 20)  labels (242400,)
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model_data/spindle_estimator', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000015403B06898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-27-07:06:33
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_data/spindle_

In [33]:
# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x":test_features},shuffle=False,num_epochs=1)

# Evaluate
results = []
for single_prediction in classifier.predict(test_input_fn):
    results.append(single_prediction['logistic'])
    
print(np.stack(results[0:10]))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_data/spindle_estimator\model.ckpt-452485
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[[0.4050245 ]
 [0.38347414]
 [0.36179665]
 [0.30687815]
 [0.30416107]
 [0.3088845 ]
 [0.33015746]
 [0.33615968]
 [0.34575698]
 [0.32066143]]


In [39]:
# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x":test_features},shuffle=False,num_epochs=1)

# Evaluate
results = []
for single_prediction in classifier.predict(test_input_fn):
    results.append(single_prediction['logistic'])
results = np.stack(results) 
np.savetxt("prediction.csv", results, delimiter=",")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_data/spindle_estimator\model.ckpt-452485
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [35]:
test_features.shape

(242400, 20)